## Demo notebook to check out the Google Places API and the results

In [18]:
import requests
from openai import OpenAI
import os

### Define functions

In [34]:
#get nearby restaurants
def get_nearby_restaurants(api_key, location, radius=200, keyword='restaurant', num_results=5):
    '''
    Returns the restaurants from the API call based on location, radius
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    params = {
        'location': location,
        'radius': radius,
        'keyword': keyword,
        'key': api_key
    }
    
    response = requests.get(base_url, params=params)
    results = response.json().get('results', [])

    # Limit the number of results to the top 'num_results'
    results = results[:num_results]
    
    return results


def get_place_details(api_key, place_id):
    '''
    Returns details on a place defined by the place_id, e.g. reviews
    '''
    base_url = 'https://maps.googleapis.com/maps/api/place/details/json'

    params = {
        'place_id': place_id,
        'key': api_key
    }

    response = requests.get(base_url, params=params)
    result = response.json().get('result', {})

    return result


def generate_summary(client, reviews):
    #Concatenate all reviews
    review_text = "\n".join(review.get('text', '') for review in reviews)

    #Define the prompt for GPT
    prompt = f"Summarize the reviews for this place:\n{review_text}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    

    #Extract the generated summary from GPT's response
    summary = response.choices[0].message.content

    return summary
    

### Set api keys from env

In [35]:
#get api keys from env file
google_api_key = os.environ.get("GOOGLE_API_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")
if google_api_key is not None:
    print("Google key found!")
if openai_api_key is not None:
    print("OpenAI key found!")

Google key found!
OpenAI key found!


### Search for places using Google Places API

In [36]:
# Munich center coordinates (for example)
munich_center = '48.1363964,11.5609501'

# Set radius and keyword
restaurants = get_nearby_restaurants(google_api_key, munich_center, radius=200, keyword='food', num_results=1)

# Print the names of the found restaurants
for restaurant in restaurants:
    #Print Restaurant name and average rating
    print("Restaurant name: ", restaurant.get('name'))
    print("Rating:  ", restaurant.get('rating'))

Restaurant name:  Subway
Rating:   4


### Generate Summary with GPT3.5

In [37]:
from openai import OpenAI


#Client, automatically searches for OPENAI_API_KEY in environment and sets as key
client = OpenAI()


#Iterate over restaurants: We want to generate a review for each one
for restaurant in restaurants:
    #Get place details 
    restaurant_id = restaurant.get('place_id')
    place_details = get_place_details(google_api_key, restaurant_id)

    #Get reviews for the place
    reviews = place_details.get('reviews', [])
        
    #Print Restaurant name
    print("Restaurant name: ", restaurant.get('name'))

    #Generate the summary
    summary = generate_summary(client, reviews)
    print(f"Summary: {summary}\n")

Restaurant name:  Subway
Summary: The time difference between London, United Kingdom, and Berlin, Germany, is currently one hour, with Berlin being one hour ahead of London.

